# EPyR Tools - Getting Started

Welcome to EPyR Tools! This notebook will guide you through the basics of loading and visualizing EPR (Electron Paramagnetic Resonance) data from Bruker spectrometers.

## What You'll Learn

- Loading EPR data from BES3T (.dsc/.dta) and ESP (.par/.spc) formats
- Understanding EPR measurement parameters
- Basic data visualization
- Converting data to open formats (CSV, JSON, HDF5)

## Requirements

Make sure you have EPyR Tools installed and sample data files in the `../data/` directory.

In [ ]:
# Import required libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add EPyR Tools to path if running from examples
epyr_root = Path().resolve().parent.parent
if str(epyr_root) not in sys.path:
    sys.path.insert(0, str(epyr_root))

import epyr.eprload as eprload
import epyr.fair as fair
from epyr.constants import CONSTANTS

print("EPyR Tools loaded successfully!")
print(f"Working directory: {Path.cwd()}")

## 1. Loading EPR Data

EPyR Tools can automatically detect and load both BES3T and ESP format files.

In [ ]:
# Find available sample files
data_dir = Path("../data")
bes3t_dir = data_dir / "BES3T"
esp_dir = data_dir / "ESP"

print("Looking for sample EPR files...")

# Check for BES3T files
bes3t_files = list(bes3t_dir.glob("*.dsc"))
esp_files = list(esp_dir.glob("*.par"))

print(f"Found {len(bes3t_files)} BES3T files and {len(esp_files)} ESP files")

if bes3t_files:
    print("BES3T files:")
    for f in bes3t_files:
        print(f"  - {f.name}")

if esp_files:
    print("ESP files:")
    for f in esp_files:
        print(f"  - {f.name}")

if not (bes3t_files or esp_files):
    print("No sample files found. Please add your Bruker EPR data files to:")
    print(f"  - {bes3t_dir} (for BES3T .dsc/.dta pairs)")
    print(f"  - {esp_dir} (for ESP .par/.spc pairs)")

In [ ]:
# Load the first available file (or create synthetic data)
sample_file = None

if bes3t_files:
    sample_file = bes3t_files[0]
    file_format = "BES3T"
elif esp_files:
    sample_file = esp_files[0]
    file_format = "ESP"

if sample_file:
    print(f"Loading {file_format} file: {sample_file.name}")
    
    # Load the EPR data
    x, y, params, filepath = eprload.eprload(str(sample_file), plot_if_possible=False)
    
    if x is not None and y is not None:
        print(f"Successfully loaded {len(x)} data points")
        print(f"Field range: {x.min():.1f} to {x.max():.1f} G")
        print(f"Signal range: {y.min():.2e} to {y.max():.2e}")
    else:
        print("Failed to load data. Creating synthetic example...")
        sample_file = None

# Create synthetic data if no real files available
if sample_file is None:
    print("Creating synthetic EPR data for demonstration...")
    
    # Generate synthetic EPR spectrum
    x = np.linspace(3200, 3400, 1000)
    
    # Create a nitroxide-like EPR signal (three lines)
    centers = [3320, 3350, 3380]  # A_N ~ 30 G splitting
    signal = np.zeros_like(x)
    
    for center in centers:
        signal += 100 * np.exp(-((x - center)**2) / 25)
    
    # Add some baseline and noise
    baseline = 0.001 * (x - 3300) + 10
    noise = np.random.normal(0, 2, len(x))
    y = signal + baseline + noise
    
    # Create mock parameters
    params = {
        'MWFQ': 9.4e9,  # 9.4 GHz
        'HCF': 3350.0,  # Center field
        'HSW': 200.0,   # Sweep width
        'RES': 1000,    # Resolution
        'AVGS': 10      # Averages
    }
    
    file_format = "Synthetic"
    print(f"Created synthetic spectrum with {len(x)} points")

## 2. Understanding EPR Parameters

Let's examine the measurement parameters extracted from the EPR file.

In [ ]:
# Display key EPR parameters
print("EPR Measurement Parameters:")
print("=" * 30)

# Parameter mapping for different file formats
param_info = {
    'MWFQ': ('Microwave Frequency', 'Hz'),
    'MF': ('Microwave Frequency', 'GHz'),
    'MWPW': ('Microwave Power', 'dB'),
    'MP': ('Microwave Power', 'W'),
    'HCF': ('Center Field', 'G'),
    'HSW': ('Sweep Width', 'G'),
    'RES': ('Resolution', 'points'),
    'REY': ('Resolution Y', 'points'),
    'AVGS': ('Number of Averages', ''),
    'SPTP': ('Sweep Time', 's'),
    'RCAG': ('Receiver Gain', 'dB'),
    'MCTC': ('Time Constant', 's')
}

for param, value in params.items():
    if param in param_info:
        name, unit = param_info[param]
        print(f"{name:20s}: {value} {unit}")
    else:
        print(f"{param:20s}: {value}")

# Calculate some derived parameters
freq_hz = params.get('MWFQ', params.get('MF', 9.4e9))
if isinstance(freq_hz, str):
    freq_hz = float(freq_hz)
    
freq_ghz = freq_hz / 1e9 if freq_hz > 1e6 else freq_hz

print("\nDerived Parameters:")
print(f"Frequency: {freq_ghz:.4f} GHz")
print(f"g-value at center: {CONSTANTS['h'] * freq_hz / (CONSTANTS['mu_B'] * params.get('HCF', 3350) * 1e-4):.4f}")
print(f"Field resolution: {params.get('HSW', 200) / len(x):.3f} G/point")

## 3. Basic Data Visualization

Let's create a professional EPR spectrum plot.

In [ ]:
# Create the main spectrum plot
plt.figure(figsize=(12, 8))

# Main spectrum
plt.subplot(2, 1, 1)
plt.plot(x, y, 'b-', linewidth=1.5, label='EPR Spectrum')
plt.xlabel('Magnetic Field (G)')
plt.ylabel('EPR Signal (a.u.)')
plt.title(f'EPR Spectrum ({file_format} format) - Frequency: {freq_ghz:.3f} GHz')
plt.grid(True, alpha=0.3)
plt.legend()

# Add parameter box
info_text = f'Center: {params.get("HCF", "N/A")} G\n'
info_text += f'Width: {params.get("HSW", "N/A")} G\n'
info_text += f'Power: {params.get("MWPW", params.get("MP", "N/A"))}'

plt.text(0.02, 0.98, info_text, transform=plt.gca().transAxes, 
         verticalalignment='top', fontsize=10,
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Derivative (first derivative is common in EPR)
plt.subplot(2, 1, 2)
dy_dx = np.gradient(y, x)
plt.plot(x, dy_dx, 'r-', linewidth=1.5, label='First Derivative')
plt.xlabel('Magnetic Field (G)')
plt.ylabel('dχ"/dB (a.u.)')
plt.title('First Derivative (Common EPR Display)')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

# Basic statistics
print("\nSpectrum Statistics:")
print(f"Mean signal: {np.mean(y):.3f}")
print(f"Signal std: {np.std(y):.3f}")
print(f"Peak-to-peak: {np.max(y) - np.min(y):.3f}")
print(f"Signal-to-noise ratio: {(np.max(y) - np.min(y)) / np.std(y):.1f}")

## 4. Converting to Open Data Formats

EPyR Tools can convert proprietary Bruker formats to open, FAIR-compliant formats.

In [ ]:
# Convert to different formats
output_dir = Path("../data/processed")
output_dir.mkdir(exist_ok=True)

base_name = "tutorial_example" if sample_file is None else sample_file.stem

print("Converting to open formats...")

# 1. CSV format - simple and universal
csv_file = output_dir / f"{base_name}.csv"
fair.to_csv(x, y, params, str(csv_file))
print(f"✓ CSV saved: {csv_file.name}")

# 2. JSON format - structured with metadata
json_file = output_dir / f"{base_name}.json"
fair.to_json(x, y, params, str(json_file))
print(f"✓ JSON saved: {json_file.name}")

# 3. HDF5 format - hierarchical and efficient
try:
    h5_file = output_dir / f"{base_name}.h5"
    fair.to_hdf5(x, y, params, str(h5_file))
    print(f"✓ HDF5 saved: {h5_file.name}")
except Exception as e:
    print(f"⚠ HDF5 export failed: {e}")
    print("  Install h5py: pip install h5py")

print(f"\nFiles saved to: {output_dir}")

In [ ]:
# Demonstrate loading the converted data back
print("Demonstrating data round-trip...")

# Load CSV back
import pandas as pd

if csv_file.exists():
    # Skip comment lines and load data
    df = pd.read_csv(csv_file, comment='#')
    x_csv = df['field_gauss'].values
    y_csv = df['intensity_au'].values
    
    print(f"Loaded {len(x_csv)} points from CSV")
    print(f"Data integrity check: {np.allclose(x, x_csv) and np.allclose(y, y_csv)}")

# Load JSON back
import json

if json_file.exists():
    with open(json_file, 'r') as f:
        json_data = json.load(f)
    
    x_json = np.array(json_data['data']['field_axis'])
    y_json = np.array(json_data['data']['intensity'])
    
    print(f"Loaded {len(x_json)} points from JSON")
    print(f"Original filename: {json_data.get('original_file', 'N/A')}")
    print(f"Microwave frequency: {json_data['measurement_parameters']['microwave_frequency']['value']:.2e} Hz")

print("\n✓ Data conversion and loading successful!")

## 5. Next Steps

Congratulations! You've learned the basics of EPyR Tools. Here's what to explore next:

### Other Tutorials:
- **02_Baseline_Correction.ipynb** - Remove baseline drift and artifacts
- **03_Advanced_Analysis.ipynb** - Peak fitting, integration, and quantitative analysis
- **04_Batch_Processing.ipynb** - Process multiple files efficiently

### Key Features to Explore:
- Baseline correction algorithms (polynomial, exponential, manual)
- Peak detection and fitting
- g-factor calculations and field calibration
- Integration and double integration for quantification
- 2D spectrum handling (field-swept, time-resolved)
- Batch processing workflows

### Adding Your Data:
1. Place BES3T files (.dsc/.dta pairs) in `examples/data/BES3T/`
2. Place ESP files (.par/.spc pairs) in `examples/data/ESP/`
3. Run this notebook again to see your real EPR data!

### Getting Help:
- Check the API documentation for detailed function descriptions
- Browse example scripts in `examples/scripts/`
- Visit the project repository for updates and issues